In [37]:
import tensorflow as tf
from zipfile import ZipFile
import os,glob
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense,MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from zipfile import ZipFile

In [38]:
from zipfile import ZipFile
import os

# Path to the zip file
zip_path = r'/Users/chengchengchen/Desktop/桌面清理大师/VSC/Sentiment_Labelled_Sentences.zip'  # Update with the correct path to combined.zip

# Extract the zip file
#with ZipFile(zip_path, 'r') as zip_ref:
#    zip_ref.extractall('/content')  # Extract to /content or any preferred directory

# Extract the zip file to a writable directory
extract_path = os.path.expanduser('~/extracted_files')  # Extract to ~/extracted_files
os.makedirs(extract_path, exist_ok=True)  # Ensure the directory exists

with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)  # Extract to the specified writable directory

print(f"Files extracted to {extract_path}")


Files extracted to /Users/chengchengchen/extracted_files


In [39]:
import os

# Path to the extracted folder
extracted_folder = os.path.expanduser('~/extracted_files/sentiment labelled sentences')

# Check and list contents
if os.path.exists(extracted_folder):
    print(os.listdir(extracted_folder))
else:
    print(f"The folder {extracted_folder} does not exist.")


['.DS_Store', 'amazon_cells_labelled.txt', 'readme.txt', 'yelp_labelled.txt', 'imdb_labelled.txt']


In [40]:
import pandas as pd
import os

# Path to the folder
folder_path = os.path.expanduser('~/extracted_files/sentiment labelled sentences')

# Files to load
files = ['imdb_labelled.txt', 'amazon_cells_labelled.txt', 'yelp_labelled.txt']

# Load all files into a single DataFrame
dataframes = []
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, sep='\t', header=None, names=['sentence', 'score'])
    df['source'] = file.split('_')[0]  # Add a source column based on file name
    dataframes.append(df)

# Combine all dataframes
data = pd.concat(dataframes, ignore_index=True)

# Display the first few rows
print(data.head())

                                            sentence  score source
0  A very, very, very slow-moving, aimless movie ...      0   imdb
1  Not sure who was more lost - the flat characte...      0   imdb
2  Attempting artiness with black & white and cle...      0   imdb
3       Very little music or anything to speak of.        0   imdb
4  The best scene in the movie was when Gerardo i...      1   imdb


In [41]:
print(data['score'].value_counts())
print(data.groupby('source')['score'].value_counts())

score
1    1386
0    1362
Name: count, dtype: int64
source  score
amazon  0        500
        1        500
imdb    1        386
        0        362
yelp    0        500
        1        500
Name: count, dtype: int64


In [42]:
import pandas as pd

# Assuming `data` is your DataFrame

# Filter the data for each score
negative_samples = data[data['score'] == 0].sample(n=1300, random_state=42)
positive_samples = data[data['score'] == 1].sample(n=200, random_state=42)

# Combine the samples
balanced_data = pd.concat([negative_samples, positive_samples])

# Shuffle the resulting dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the new distribution
print(balanced_data['score'].value_counts())


score
0    1300
1     200
Name: count, dtype: int64


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine all files into one DataFrame
for file in files:
    file_path = os.path.expanduser(f'{folder_path}/{file}')
    df = pd.read_csv(file_path, sep='\t', header=None, names=['sentence', 'score'])
    dataframes.append(df)

data = pd.concat(dataframes, ignore_index=True)

# Preprocess the text
tfidf = TfidfVectorizer(stop_words='english', max_features=200)  # Limit features to 5000
X = tfidf.fit_transform(balanced_data['sentence'])
y = balanced_data['score']

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
y_train = y_train.astype(int)

In [46]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

# Soft-margin SVM using CVXPY
def soft_margin_svm(X, y, C):
    """
    Solves the soft-margin SVM problem using CVXPY
    Args:
    - X: Data matrix of shape (m, n) where m is the number of points and n is the dimension (features).
    - y: Labels vector of shape (m,) with entries in {-1, 1}.
    - C: Regularization parameter.
    Returns:
    - b: Coefficient vector of shape (n,)
    - c: Intercept (scalar)
    - xi: Slack variables vector of shape (m,)
    """
    m, n = X.shape

    # Define the variables
    b = cp.Variable(n)    # weight vector
    c = cp.Variable()     # bias
    xi = cp.Variable(m)   # slack variables

    # Define the objective function: minimize (1/2) * ||b||^2 + C * sum(xi)
    objective = cp.Minimize(0.5 * cp.sum_squares(b) + C * cp.sum(xi))

    # Define the constraints: y_i * (b^T x_i + c) >= 1 - xi_i, xi_i >= 0
    constraints = [y[i] * (X[i] @ b + c) >= 1 - xi[i] for i in range(m)]
    constraints += [xi >= 0]

    # Solve the problem
    prob = cp.Problem(objective, constraints)
    prob.solve()

    # Return the solution: b, c, and xi
    return b.value, c.value, xi.value

# Function to predict using the learned SVM model
def predict(X, b, c):
    return np.sign(X @ b + c)


In [47]:
import numpy as np
# Convert labels in y_train to 1 and -1
# 1 is postive, -1 is negative
y_train = np.where(y_train == 0, -1, y_train)  # Replace 0 with -1



In [48]:
from sklearn.metrics import accuracy_score, classification_report
# Run the SVM (Soft-margin SVM)
C = 1.0  # Regularization parameter
b_opt, c_opt, xi_opt = soft_margin_svm(X_train, y_train, C)

# Predict the labels on the test set
y_pred_train = predict(X_train, b_opt, c_opt)

y_pred_test = predict(X_test, b_opt, c_opt)

# Compute accuracy and display classification report
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

print("Classification Report for Training Set:")
print(classification_report(y_train, y_pred_train))

Training Accuracy: 88.42%
Classification Report for Training Set:
              precision    recall  f1-score   support

          -1       0.88      1.00      0.94      1033
           1       0.89      0.19      0.32       167

    accuracy                           0.88      1200
   macro avg       0.89      0.59      0.63      1200
weighted avg       0.88      0.88      0.85      1200



In [49]:
def USVM(A, B, n_universum, C=1.0, C_u=1.0, epsilon=0.1):
    """
    Universum SVM for any binary classification task.

    Args:
    - A: Matrix of points for class 1 (positive class).
    - B: Matrix of points for class -1 (negative class).
    - n_universum: Number of universum points to generate.
    - C: Regularization parameter for training data.
    - C_u: Regularization parameter for universum data.
    - epsilon: Margin for universum points.

    Returns:
    - b_opt: Optimal weight vector.
    - c_opt: Optimal bias term.
    - universum_points: Generated universum points.
    """

    n_class1, n_features = A.shape
    n_class_minus_1 = B.shape[0]

    # Generate Universum points by averaging randomly selected points from both classes
    random_indices_class_1 = np.random.choice(n_class1, n_universum, replace=True)
    random_indices_class_minus_1 = np.random.choice(n_class_minus_1, n_universum, replace=True)
    universum_points = (A[random_indices_class_1] + B[random_indices_class_minus_1]) / 2

    # Combine the classes into a single dataset
    X_train = np.vstack((A, B))
    y_train = np.hstack((np.ones(n_class1), -np.ones(n_class_minus_1)))

    # Variables for optimization
    b = cp.Variable(n_features)  # We use 'b' for the weight vector
    c = cp.Variable()
    xi = cp.Variable(n_class1 + n_class_minus_1)  # Slack variables for misclassification
    psi = cp.Variable(n_universum)  # Slack variables for universum points (positive side)
    phi = cp.Variable(n_universum)  # Slack variables for universum points (negative side)

    # Constraints
    constraints = []

    # For class 1 (label +1)
    for i in range(n_class1):
        constraints.append(A[i, :] @ b + c >= 1 - xi[i])

    # For class -1 (label -1)
    for i in range(n_class_minus_1):
        constraints.append(B[i, :] @ b + c <= -1 + xi[n_class1 + i])

    # Universum constraints
    for j in range(n_universum):
        constraints.append(universum_points[j, :] @ b + c <= epsilon + psi[j])
        constraints.append(universum_points[j, :] @ b + c >= -epsilon - phi[j])

    # Non-negative slack variables
    constraints += [xi >= 0, psi >= 0, phi >= 0]

    # Objective function: minimize the sum of errors and regularization term
    objective = cp.Minimize(0.5 * cp.norm(b, 2)**2 + C * cp.sum(xi) + C_u * (cp.sum(psi) + cp.sum(phi)))

    # Solve the optimization problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    # Get the optimal values of b and c
    b_opt = b.value
    c_opt = c.value

    return b_opt, c_opt, universum_points

def predict(X, b, c):
    """
    Predict the labels based on the decision boundary.

    Args:
    - X: Data points.
    - b: Optimal weight vector.
    - c: Optimal bias term.

    Returns:
    - Predictions (1 or -1).
    """
    return np.sign(X @ b + c)


In [50]:
from sklearn.metrics import classification_report

A = X_train[y_train == 1]  
B = X_train[y_train == -1]
n_universum = 1100

# Run the Universum SVM
b_opt, c_opt, universum_points = USVM(A, B, n_universum, C =1, C_u =1, epsilon=0.1)


# Predict the labels based on the learned decision boundary
y_pred = predict(X_train, b_opt, c_opt)

# Compute the accuracy score
accuracy = accuracy_score(y_train, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Display the classification report
print("Classification Report:")
print(classification_report(y_train, y_pred))

Accuracy: 86.17%
Classification Report:
              precision    recall  f1-score   support

          -1       0.93      0.91      0.92      1033
           1       0.50      0.57      0.54       167

    accuracy                           0.86      1200
   macro avg       0.72      0.74      0.73      1200
weighted avg       0.87      0.86      0.87      1200



In [51]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

def tsvm(A, B, C1=1.0, C2=1.0):
    """
    TSVM

    Args:
    - A: Matrix of points for class 1 (positive class).
    - B: Matrix of points for class -1 (negative class).
    - n_universum: Number of Universum points to generate.
    - C1: Regularization parameter for class 1.
    - C2: Regularization parameter for class -1.

    Returns:
    - b1_opt, c1_opt: Optimal weight vector and bias for class 1 hyperplane.
    - b2_opt, c2_opt: Optimal weight vector and bias for class -1 hyperplane.
    """

    n_class1, n_features = A.shape
    n_class_minus_1 = B.shape[0]



    # Optimization variables for Class 1
    b1 = cp.Variable(n_features)
    c1 = cp.Variable()
    xi1 = cp.Variable(n_class_minus_1)  # Slack variables for class -1 constraints
    
    # Optimization variables for Class 2
    b2 = cp.Variable(n_features)
    c2 = cp.Variable()
    xi2 = cp.Variable(n_class1)  # Slack variables for class 1 constraints
  

    # Objective function for Class 1
    objective1 = cp.Minimize(0.5 * cp.norm(A @ b1 + c1, 2)**2 + C1 * cp.sum(xi1))

    # Constraints for Class 1
    constraints1 = [
        -B @ b1 - c1 + xi1 >= np.ones(n_class_minus_1),
        xi1 >= 0  # Class -1 points
    ]

    # Solve for Class 1
    problem1 = cp.Problem(objective1, constraints1)
    problem1.solve()
    b1_opt = b1.value
    c1_opt = c1.value

    # Objective function for Class 2
    objective2 = cp.Minimize(0.5 * cp.norm(B @ b2 + c2, 2)**2 + C2 * cp.sum(xi2))

    # Constraints for Class 2
    constraints2 = [
        A @ b2 + c2 + xi2 >= np.ones(n_class1),
        xi2 >= 0   # Class 1 points
    ]

    # Solve for Class 2
    problem2 = cp.Problem(objective2, constraints2)
    problem2.solve()
    b2_opt = b2.value
    c2_opt = c2.value

    return b1_opt, c1_opt, b2_opt, c2_opt

def predict(X, b1_opt, c1_opt, b2_opt, c2_opt):
    """
    Predicts the labels for the given data points using TSVM with learned hyperplanes.

    Args:
    - X: Input data points, shape (n_samples, n_features).
    - b1_opt: Optimal weight vector for class 1 hyperplane.
    - c1_opt: Optimal bias for class 1 hyperplane.
    - b2_opt: Optimal weight vector for class -1 hyperplane.
    - c2_opt: Optimal bias for class -1 hyperplane.

    Returns:
    - y_pred: Predicted labels for the data points, +1 or -1.
    """
    # Calculate distance from hyperplane 1 (class +1)
    distance_to_class1 = np.abs(np.dot(X, b1_opt) + c1_opt)
    
    # Calculate distance from hyperplane 2 (class -1)
    distance_to_class2 = np.abs(np.dot(X, b2_opt) + c2_opt)
    
    # Assign label based on which hyperplane is closer
    y_pred = np.where(distance_to_class1 < distance_to_class2, 1, -1)
    
    return y_pred

In [52]:
from sklearn.metrics import classification_report
np.random.seed(42)

A = X_train[y_train == 1]  
B = X_train[y_train == -1]

In [53]:
# Train tSVM
b1_opt, c1_opt, b2_opt, c2_opt  = tsvm(
    A, B, C1=1.0, C2=1.0)

X_train_dense = X_train.toarray()  # Convert to dense format

# Predict on the train data
y_pred = predict(X_train_dense, b1_opt, c1_opt, b2_opt, c2_opt)

# Calculate the accuracy
accuracy = accuracy_score(y_train, y_pred)

# Output the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

# Display the classification report
print("Classification Report:")
print(classification_report(y_train, y_pred))

Accuracy: 90.00%
Classification Report:
              precision    recall  f1-score   support

          -1       0.90      0.99      0.94      1033
           1       0.87      0.33      0.48       167

    accuracy                           0.90      1200
   macro avg       0.89      0.66      0.71      1200
weighted avg       0.90      0.90      0.88      1200



In [54]:
from scipy.sparse import issparse
def UTSVM(X, y, n_universum=25, C1=1.0, C2=1.0, Cu=1.0, epsilon=0.1):
    """
    Universum Twin SVM.

    Args:
    - X: Training matrix
    - y: Labels vector
    - n_universum: Number of Universum points to generate.
    - C1: Regularization parameter for class 1.
    - C2: Regularization parameter for class -1.
    - Cu: Regularization parameter for Universum points.
    - epsilon: Margin for Universum points.

    Returns:
    - b1_opt, c1_opt: Optimal weight vector and bias for class 1 hyperplane.
    - b2_opt, c2_opt: Optimal weight vector and bias for class -1 hyperplane.
    """


    # A: Matrix of points for class 1 (positive class)
    #A = X[y==1]
    # B: Matrix of points for class -1 (negative class)
    #B = X[y==-1]

    n_class1, n_features = A.shape
    n_class_minus_1 = B.shape[0]


    # Generate Universum points by averaging randomly selected points from both classes
    random_indices_class_1 = np.random.choice(n_class1, n_universum, replace=True)
    random_indices_class_minus_1 = np.random.choice(n_class_minus_1, n_universum, replace=True)
    U = (A[random_indices_class_1] + B[random_indices_class_minus_1]) / 2

    # Optimization variables for Class 1
    b1 = cp.Variable(n_features)
    c1 = cp.Variable()
    xi1 = cp.Variable(n_class_minus_1)  # Slack variables for class -1 constraints
    psi1 = cp.Variable(n_universum)  # Slack variables for Universum points (class 1)

    # Optimization variables for Class 2
    b2 = cp.Variable(n_features)
    c2 = cp.Variable()
    xi2 = cp.Variable(n_class1)  # Slack variables for class 1 constraints
    psi2 = cp.Variable(n_universum)  # Slack variables for Universum points (class -1)

    # Objective function for Class 1
    objective1 = cp.Minimize(0.5 * cp.norm(A @ b1 + c1, 2)**2 + C1 * cp.sum(xi1) + Cu * cp.sum(psi1))

    # Constraints for Class 1
    constraints1 = [
        -B @ b1 - c1 + xi1 >= np.ones(n_class_minus_1),  # Class -1 points
        U @ b1 + c1 + psi1 >= (-1 + epsilon) * np.ones(n_universum),  # Universum points
        xi1 >= 0, psi1 >= 0  # Non-negative slack variables
    ]

    # Solve for Class 1
    problem1 = cp.Problem(objective1, constraints1)
    problem1.solve()
    b1_opt = b1.value
    c1_opt = c1.value

    # Objective function for Class 2
    objective2 = cp.Minimize(0.5 * cp.norm(B @ b2 + c2, 2)**2 + C2 * cp.sum(xi2) + Cu * cp.sum(psi2))

    # Constraints for Class 2
    constraints2 = [
        A @ b2 + c2 + xi2 >= np.ones(n_class1),  # Class 1 points
        -U @ b2 - c2 + psi2 >= (-1 + epsilon) * np.ones(n_universum),  # Universum points
        xi2 >= 0, psi2 >= 0  # Non-negative slack variables
    ]

    # Solve for Class 2
    problem2 = cp.Problem(objective2, constraints2)
    problem2.solve()
    b2_opt = b2.value
    c2_opt = c2.value

    return b1_opt, c1_opt, b2_opt, c2_opt, U

def predict(X, b1_opt, c1_opt, b2_opt, c2_opt):
    """
    Predicts the labels for the given data points using TSVM with learned hyperplanes.

    Args:
    - X: Input data points, shape (n_samples, n_features).
    - b1_opt: Optimal weight vector for class 1 hyperplane.
    - c1_opt: Optimal bias for class 1 hyperplane.
    - b2_opt: Optimal weight vector for class -1 hyperplane.
    - c2_opt: Optimal bias for class -1 hyperplane.

    Returns:
    - y_pred: Predicted labels for the data points, +1 or -1.
    """
    # Calculate distance from hyperplane 1 (class +1)
    distance_to_class1 = np.abs(np.dot(X, b1_opt) + c1_opt)
    
    # Calculate distance from hyperplane 2 (class -1)
    distance_to_class2 = np.abs(np.dot(X, b2_opt) + c2_opt)
    
    # Assign label based on which hyperplane is closer
    y_pred = np.where(distance_to_class1 < distance_to_class2, 1, -1)
    
    return y_pred




In [55]:
# UTSVM
n_universum=1100

b1_opt, c1_opt, b2_opt, c2_opt, U  = UTSVM(
    A, B, n_universum, C1=1.0, C2=1.0, Cu=1.0)

# Predict on the train data
y_pred = predict(X_train_dense, b1_opt, c1_opt, b2_opt, c2_opt)

# Calculate the accuracy
accuracy = accuracy_score(y_train, y_pred)

# Output the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

# Display the classification report
print("Classification Report:")
print(classification_report(y_train, y_pred))

Accuracy: 89.17%
Classification Report:
              precision    recall  f1-score   support

          -1       0.90      0.98      0.94      1033
           1       0.72      0.37      0.48       167

    accuracy                           0.89      1200
   macro avg       0.81      0.67      0.71      1200
weighted avg       0.88      0.89      0.88      1200



In [56]:
from sklearn.base import BaseEstimator, ClassifierMixin

class QSVM(BaseEstimator, ClassifierMixin):
    def __init__(self, mu=0.2):
        self.mu = mu

    def fit(self, X, y):
        self.d = self._calculate_d(X.shape[1])  # Calculate dimensionality once
        G, r = self.compute_G_and_r(X)
        self.z, self.c = self.optimize_z_c(G, r, y, self.mu)
        return self  # Return self instead of z and c

    def predict(self, X):
        r = self._compute_r_matrix(X)
        return np.sign(r @ self.z + self.c)

    def score(self, X, y):
        predictions = self.predict(X)
        return accuracy_score(y, predictions)

    def compute_G_and_r(self, X):
        """Compute the matrices G and r based on X."""
        m, n = X.shape
        G = np.zeros((self.d, self.d))
        r = np.zeros((m, self.d))
        D_n = self.get_duplication_matrix(n)

        print("Start computing G and r...")
        for i in tqdm(range(m)):
            x_i = X[i, :]
            s_i = self._compute_s_i(x_i)
            r[i] = np.concatenate([s_i, x_i])
            X_i = np.kron(np.eye(n), x_i.reshape(1, -1))
            M_i = X_i @ D_n
            H_i = np.hstack([M_i, np.eye(n)])
            G += 2 * H_i.T @ H_i
        print("Finished computing G and r...\n")

        return G, r

    def loss(self, z, G, r, y, c, mu):
        square_loss = 0.5 * z.T @ G @ z
        y_hat = r @ z + c
        hinge_loss = np.sum(np.maximum(0, 1 - np.multiply(y, y_hat)))
        return square_loss + mu * hinge_loss

    def optimize_z_c(self, G, r, y, mu):
        d = G.shape[0]
        z = cp.Variable((d, 1))
        c = cp.Variable()
        y = np.array(y).reshape(-1, 1)

        square_loss = 0.5 * cp.quad_form(z, G)
        y_hat = r @ z + c
        hinge_loss = cp.sum(cp.maximum(0, 1 - cp.multiply(y, y_hat)))
        objective = cp.Minimize(square_loss + mu * hinge_loss)
        prob = cp.Problem(objective)
        result = prob.solve(solver=cp.CLARABEL, verbose=False)

        return z.value, c.value

    def _compute_r_matrix(self, X):
        """Helper to compute the r matrix for predictions."""
        m, n = X.shape
        r = np.zeros((m, self.d))
        for i in range(m):
            x_i = X[i, :]
            s_i = self._compute_s_i(x_i)
            r[i] = np.concatenate([s_i, x_i])
        return r

    def _compute_s_i(self, x_i):
        """Helper to compute s_i (half-vectorization) from x_i."""
        x_i_x_i_T = np.outer(x_i, x_i)
        return 0.5 * self.hvec(x_i_x_i_T)

    def hvec(self, A):
        """Half-vectorization of a symmetric matrix A."""
        indices = np.triu_indices(A.shape[0])
        return A[indices]

    def get_duplication_matrix(self, n):
        """Duplication matrix for vectorizing a symmetric matrix."""
        i_indices = np.arange(n*n)
        j_indices = np.zeros((n, n), dtype=int)

        z = 0
        for x in range(n):
            for y in range(x, n):
                j_indices[x, y] = j_indices[y, x] = z
                z += 1

        j_indices_flat = j_indices.flatten()
        data = np.ones(n*n, dtype=int)
        return coo_matrix((data, (i_indices, j_indices_flat)), shape=(n*n, (n*(n+1))//2))

    def _calculate_d(self, n):
        """Helper to calculate the dimensionality once."""
        return n + n * (n + 1) // 2


In [57]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Normalize data (important for SVM)
scaler_standard = StandardScaler()
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

scaler_standard = StandardScaler()
X_train_standard = scaler_standard.fit_transform(X_train_dense)
X_test_standard = scaler_standard.transform(X_test_dense)


pca = PCA()
X_pca = pca.fit_transform(X_train_standard)
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = explained_variance_ratio.cumsum()
k = (cumulative_variance>=0.9).argmax()+1

pca= PCA(n_components=k)
X_train_reduced = pca.fit_transform(X_train_standard)
X_test_reduced = pca.transform(X_test_standard)

print(f"Training data shape: {X_train_reduced.shape}")
print(f"Test data shape: {X_test_reduced.shape}")

Training data shape: (1200, 164)
Test data shape: (300, 164)


In [58]:
from scipy.sparse import coo_matrix
from tqdm import tqdm

qsvm = QSVM(mu = 2)
clf = qsvm.fit(X_train_reduced, y_train)

Start computing G and r...


100%|██████████| 1200/1200 [07:47<00:00,  2.57it/s]


Finished computing G and r...



In [59]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the test data
y_pred = qsvm.predict(X_train_reduced)

# Calculate the accuracy
accuracy = accuracy_score(y_train, y_pred)

# Output the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

# Display the classification report
print("Classification Report:")
print(classification_report(y_train, y_pred))

Accuracy: 92.00%
Classification Report:
              precision    recall  f1-score   support

          -1       0.92      1.00      0.96      1033
           1       0.95      0.45      0.61       167

    accuracy                           0.92      1200
   macro avg       0.93      0.72      0.78      1200
weighted avg       0.92      0.92      0.91      1200



In [60]:
class UQSVM:
    def __init__(self, C=1, C_u=1, epsilon=0.1, n_universum=10):

        """
        Universum Quadratic SVM for any binary classification task.
        Args:

        - C: Regularization parameter for training data.
        - C_u: Regularization parameter for universum data.
        - epsilon: Margin for universum points.
        - n_universum: Number of universum points to generate.
        """
        self.C = C
        self.C_u = C_u
        self.epsilon = epsilon
        self.n_universum = n_universum

    def fit(self, X, y):
        """
        Fit the U-QSVM model.

        X: Training data matrix.
        y: Labels corresponding to rows of X.
        """
        self.d = self._calculate_d(X.shape[1])  # Calculate dimensionality once

        # Separate positive and negative samples
        A = X[y == 1]
        B = X[y == -1]

        # Generate Universum points by averaging randomly selected points from both classes
        n_class1, n_features = A.shape
        n_class_minus_1 = B.shape[0]
        random_indices_class_1 = np.random.choice(n_class1, self.n_universum, replace=True)
        random_indices_class_minus_1 = np.random.choice(n_class_minus_1, self.n_universum, replace=True)
        U = (A[random_indices_class_1] + B[random_indices_class_minus_1]) / 2

        # Compute G, r, and r_u matrices
        G, r, r_u = self.compute_G_and_r_and_r_u(X, U)
        self.z, self.c = self.optimize_z_c(G, r, y, r_u)

        return self.z, self.c, U

    def predict(self, X):
        """
        Make predictions using the fitted U-QSVM model.

        X: Test data (features).
        """
        r = self._compute_r_matrix(X)
        return np.sign(r @ self.z + self.c)

    def score(self, X, y):
        """
        Calculate the accuracy of the U-QSVM model.

        X: Test data (features).
        y: True labels for the test data.
        """
        predictions = self.predict(X)
        return accuracy_score(y, predictions)

    def compute_G_and_r_and_r_u(self, X, U):
        """Compute the matrices G, r, and r_u based on trainsing matrix X and Universum matrix U."""
        m, n = X.shape
        G = np.zeros((self.d, self.d))
        r = np.zeros((m, self.d))
        D_n = self.get_duplication_matrix(n)

        R, _ = U.shape
        r_u = np.zeros((R, self.d))

        print("Start computing G and r...")
        for i in tqdm(range(m)):
            x_i = X[i, :]
            s_i = self._compute_s_i(x_i)
            r[i] = np.concatenate([s_i, x_i])
            X_i = np.kron(np.eye(n), x_i.reshape(1, -1))
            M_i = X_i @ D_n
            H_i = np.hstack([M_i, np.eye(n)])
            G += 2 * H_i.T @ H_i
        print("Finished computing G and r...")

        print("Start computing r_u...")
        for i in tqdm(range(R)):
            u_i = U[i, :]
            s_u_i = self._compute_s_i(u_i)
            r_u[i] = np.concatenate([s_u_i, u_i])
        print("Finished computing r_u...")

        return G, r, r_u

    def loss(self, z, G, r, y, c, r_u):
        """Compute the objective function for the U-QSVM model."""
        square_loss = 0.5 * z.T @ G @ z
        hinge_loss = np.sum(np.maximum(0, 1 - np.multiply(y, r @ z + c)))
        hinge_loss_Universum_1 = np.sum(np.maximum(0,  - self.epsilon + (r_u @ z + c)))
        hinge_loss_Universum_2 = np.sum(np.maximum(0,  - self.epsilon - (r_u @ z + c)))
        return square_loss + self.C * hinge_loss + self.C_u * (hinge_loss_Universum_1 + hinge_loss_Universum_2)

    def optimize_z_c(self, G, r, y, r_u):
        """Optimize the parameters z and c using convex programming."""
        d = G.shape[0]
        z = cp.Variable((d, 1))
        c = cp.Variable()
        y = np.array(y).reshape(-1, 1)

        square_loss = 0.5 * cp.quad_form(z, G)
        hinge_loss = cp.sum(cp.maximum(0, 1 - cp.multiply(y, r @ z + c)))
        hinge_loss_Universum_1 = cp.sum(cp.maximum(0,  - self.epsilon - (r_u @ z + c)))
        hinge_loss_Universum_2 = cp.sum(cp.maximum(0,  - self.epsilon + (r_u @ z + c)))
        objective = cp.Minimize(square_loss + self.C * hinge_loss + self.C_u * (hinge_loss_Universum_1 + hinge_loss_Universum_2))
        prob = cp.Problem(objective)
        result = prob.solve(solver=cp.SCS, verbose=False)  # Change solver if needed

        return z.value, c.value

    def _compute_r_matrix(self, X):
        """Helper to compute the r matrix for predictions."""
        m, n = X.shape
        r = np.zeros((m, self.d))
        for i in range(m):
            x_i = X[i, :]
            s_i = self._compute_s_i(x_i)
            r[i] = np.concatenate([s_i, x_i])
        return r

    def _compute_s_i(self, x_i):
        """Helper to compute s_i (half-vectorization) from x_i."""
        x_i_x_i_T = np.outer(x_i, x_i)
        return 0.5 * self.hvec(x_i_x_i_T)

    def hvec(self, A):
        """Half-vectorization of a symmetric matrix A."""
        indices = np.triu_indices(A.shape[0])
        return A[indices]

    def get_duplication_matrix(self, n):
        """Duplication matrix for vectorizing a symmetric matrix."""
        i_indices = np.arange(n * n)
        j_indices = np.zeros((n, n), dtype=int)

        z = 0
        for x in range(n):
            for y in range(x, n):
                j_indices[x, y] = j_indices[y, x] = z
                z += 1

        j_indices_flat = j_indices.flatten()
        data = np.ones(n * n, dtype=int)
        return coo_matrix((data, (i_indices, j_indices_flat)), shape=(n * n, (n * (n + 1)) // 2))

    def _calculate_d(self, n):
        """Helper to calculate the dimensionality once."""
        return n + n * (n + 1) // 2


In [61]:
uqsvm = UQSVM(C=1.0, C_u=0.5, epsilon=0.3, n_universum=1289)
clf = uqsvm.fit(X_train_reduced, y_train)

Start computing G and r...


100%|██████████| 1200/1200 [07:30<00:00,  2.67it/s]


Finished computing G and r...
Start computing r_u...


100%|██████████| 1289/1289 [00:00<00:00, 9494.07it/s]


Finished computing r_u...


In [62]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the test data
y_pred = uqsvm.predict(X_train_reduced)

# Calculate the accuracy
accuracy = accuracy_score(y_train, y_pred)

# Output the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

# Display the classification report
print("Classification Report:")
print(classification_report(y_train, y_pred))

Accuracy: 91.33%
Classification Report:
              precision    recall  f1-score   support

          -1       0.91      0.99      0.95      1033
           1       0.91      0.42      0.57       167

    accuracy                           0.91      1200
   macro avg       0.91      0.71      0.76      1200
weighted avg       0.91      0.91      0.90      1200



In [63]:
class QTSVM:
    def __init__(self, C_1=1, C_2=1):

        """
        Imbalanced Universum Quadratic Twin SVM for any binary classification task.
        For each class, this model finds a quadratic function that passes through that class
        but makes sure the other class stay away one unit! Universum points are generated
        depending on the class that we are handling.

        Args:
        - C_1: Regularization parameter for class 1 points.
        - C_2: Regularization parameter for class -1 points.
        """

        self.C_1 = C_1
        self.C_2 = C_2

    def fit(self, X, y):
        """
        Fit the ImUQTSVM model.

        X: Training data matrix.
        y: Labels corresponding to rows of X.
        """
        self.d = self._calculate_d(X.shape[1])  # Calculate dimensionality once

        # Separate positive and negative samples
        A = X[y == 1]
        B = X[y == -1]

        # Undersample class -1 to construct a reduced majority matrix
        m_1 = A.shape[0]
        B_reduced_indices = np.random.choice(B.shape[0], m_1, replace=False)
        B_reduced = B[B_reduced_indices]

        m_2 = B.shape[0]
        r = m_2 - m_1
        # Create A_bar: Randomly choose rows from A with replacement
        A_bar_indices = np.random.choice(m_1, r, replace=True)
        A_bar = A[A_bar_indices]

        # Create B_bar: Randomly choose rows from B without replacement
        B_bar_indices = np.random.choice(m_2, r, replace=False)
        B_bar = B[B_bar_indices]

        # Compute r matrices
        r_1 = self.compute_r(A)
        r_2 = self.compute_r(B)
        r_2_reduced = self.compute_r(B_reduced)


        self.z_1, self.c_1 = self.optimize_z_1_c_1(r_1, r_2_reduced)
        self.z_2, self.c_2 = self.optimize_z_2_c_2(r_2, r_1)

        return self.z_1, self.c_1, self.z_2, self.c_2

    def predict(self, X, z_1, c_1, z_2, c_2):
        """
        Make predictions using the fitted UQTSVM model.
        X: Test data (features).
        """
        y_hat = np.ones(X.shape[0])
        for i in range(X.shape[0]):
            r_i = self.compute_r(X[i,:].reshape(1,-1))
            H_i = self.compute_H(X[i,:])

            numerator_1 = np.abs(r_i @ z_1 + c_1)
            denominator_1 = np.linalg.norm(H_i @ z_1 + c_1) ** 2
            numerator_2 = np.abs(r_i @ z_2 + c_2)
            denominator_2 = np.linalg.norm(H_i @ z_2 + c_2) ** 2

            expression_1 = numerator_1 / denominator_1
            expression_2 = numerator_2 / denominator_2

            if expression_1 >= expression_2:
                y_hat[i] = -1

        return y_hat

    def score(self, X, y):
        """
        Calculate the accuracy of the U-QSVM model.

        X: Test data (features).
        y: True labels for the test data.
        """
        #predictions = self.predict(X, z_1, c_1, z_2, c_2)
        predictions = self.predict(X, self.z_1, self.c_1, self.z_2, self.c_2)

        return accuracy_score(y, predictions)


    def compute_H(self, x):
      n = x.shape[0]
      s_x = self.compute_s_i(x)
      r_x = np.concatenate([s_x, x])
      X_x = np.kron(np.eye(n), x.reshape(1, -1))
      D_n = self.get_duplication_matrix(n)
      M_x = X_x @ D_n
      H_x = np.hstack([M_x, np.eye(n)])
      return H_x


    def optimize_z_1_c_1(self, r_1, r_2):

        d = r_1.shape[1]
        z_1 = cp.Variable((d, 1))
        c_1 = cp.Variable()

        # Square loss for regression points
        square_loss_1 = cp.sum_squares(r_1 @ z_1 + c_1)

        # Slack variables for errors in class 2 and Universum
        slack_class_2 = cp.Variable((r_2.shape[0], 1), nonneg=True)

        # slack_class_2 = cp.Variable(r_2.shape[0], nonneg=True)
        # slack_universum = cp.Variable(r_u.shape[0], nonneg=True)

        # Constraints
        constraints = []

        # Ensure all points in class 2 satisfy the margin or account for errors
        constraints.append(r_2 @ z_1 + c_1 <= -1 + slack_class_2)

        # Objective: Minimize square loss + hinge penalties for class 2 and Universum
        objective = cp.Minimize(
            square_loss_1
            + self.C_1 * cp.sum(slack_class_2)
        )

        # Problem definition
        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.SCS, verbose=False)  # Use any preferred solver

        return z_1.value, c_1.value


    def optimize_z_2_c_2(self, r_2, r_1):

        d = r_2.shape[1]
        z_2 = cp.Variable((d, 1))
        c_2 = cp.Variable()

        # Square loss for class 1 points
        square_loss_1 = cp.sum_squares(r_2 @ z_2 + c_2)

        # Slack variables for errors in class 1 and Universum
        slack_class_1 = cp.Variable((r_1.shape[0], 1), nonneg=True)

        # slack_class_1 = cp.Variable(r_1.shape[0], nonneg=True)
        # slack_universum = cp.Variable(r_u.shape[0], nonneg=True)

        # Constraints
        constraints = []

        # Ensure all points in class 1 satisfy the margin or account for errors
        constraints.append(r_1 @ z_2 + c_2 >= 1 - slack_class_1)


        # Objective: Minimize square loss + hinge penalties for class 1 and Universum
        objective = cp.Minimize(
            square_loss_1
            + self.C_2 * cp.sum(slack_class_1)
        )

        # Problem definition
        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.SCS, verbose=False)  # Use any preferred solver

        return z_2.value, c_2.value

    def compute_r(self, X):
        """Helper to compute the r matrix for predictions."""
        m, n = X.shape
        r = np.zeros((m, self.d))
        for i in range(m):
            x_i = X[i, :]
            s_i = self.compute_s_i(x_i)
            r[i] = np.concatenate([s_i, x_i])
        return r

    def compute_s_i(self, x_i):
        """Helper to compute s_i (half-vectorization) from x_i."""
        x_i_x_i_T = np.outer(x_i, x_i)
        return 0.5 * self.hvec(x_i_x_i_T)

    def hvec(self, A):
        """Half-vectorization of a symmetric matrix A."""
        indices = np.triu_indices(A.shape[0])
        return A[indices]

    def get_duplication_matrix(self, n):
        """Duplication matrix for vectorizing a symmetric matrix."""
        i_indices = np.arange(n * n)
        j_indices = np.zeros((n, n), dtype=int)

        z = 0
        for x in range(n):
            for y in range(x, n):
                j_indices[x, y] = j_indices[y, x] = z
                z += 1

        j_indices_flat = j_indices.flatten()
        data = np.ones(n * n, dtype=int)
        return coo_matrix((data, (i_indices, j_indices_flat)), shape=(n * n, (n * (n + 1)) // 2))

    def _calculate_d(self, n):
        """Helper to calculate the dimensionality once."""
        return n + n * (n + 1) // 2


In [64]:
np.random.seed(42)

C_1 = 1
C_2 = 1

# Step 2: Initialize and train the UQTSVM model (Assuming the class UQTSVM is already defined)
qtsvm = QTSVM(C_1 = C_1, C_2 = C_2)
z_1, c_1, z_2, c_2 = qtsvm.fit(X_train_reduced, y_train)

In [65]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the test data
y_pred = qtsvm.predict(X_train_reduced, z_1, c_1, z_2, c_2)

# Calculate the accuracy
accuracy = accuracy_score(y_train, y_pred)

# Output the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

# Display the classification report
print("Classification Report:")
print(classification_report(y_train, y_pred))

Accuracy: 93.42%
Classification Report:
              precision    recall  f1-score   support

          -1       0.96      0.96      0.96      1033
           1       0.77      0.76      0.76       167

    accuracy                           0.93      1200
   macro avg       0.86      0.86      0.86      1200
weighted avg       0.93      0.93      0.93      1200



In [66]:
import math

class UQTSVM:
    def __init__(self, C_1=1, C_2=1, C_u=1, epsilon=0.15):

        """
        Imbalanced Universum Quadratic Twin SVM for any binary classification task.
        For each class, this model finds a quadratic function that passes through that class
        but makes sure the other class stay away one unit! Universum points are generated
        depending on the class that we are handling.

        Args:
        - C_1: Regularization parameter for class 1 points.
        - C_2: Regularization parameter for class -1 points.
        - C_u: Regularization parameter for Universum data.
        - epsilon: Margin for universum points.
        """

        self.C_1 = C_1
        self.C_2 = C_2
        self.C_u = C_u
        self.epsilon = epsilon

    def fit(self, X, y, n_universum=1289):
        """
        Fit the ImUQTSVM model.

        X: Training data matrix.
        y: Labels corresponding to rows of X.
        """
        self.d = self._calculate_d(X.shape[1])  # Calculate dimensionality once

        # Separate positive and negative samples
        A = X[y == 1]
        B = X[y == -1]

        # Undersample class -1 to construct a reduced majority matrix
        m_1 = A.shape[0]
        B_reduced_indices = np.random.choice(B.shape[0], m_1, replace=False)
        B_reduced = B[B_reduced_indices]

        m_2 = B.shape[0]
        r = m_2 - m_1
        # Create A_bar: Randomly choose rows from A with replacement
        A_bar_indices = np.random.choice(m_1, r, replace=True)
        A_bar = A[A_bar_indices]

        # Create B_bar: Randomly choose rows from B without replacement
        B_bar_indices = np.random.choice(m_2, r, replace=False)
        B_bar = B[B_bar_indices]

        # Generate Universum points
        U = (A_bar + B_bar) / 2  # Initial Universum points
        g = math.ceil(m_1 // 2)  # Reduced Universum size for class 1
        U_reduced = U[np.random.choice(U.shape[0], g, replace=True)]

        # Compute r matrices
        r_1 = self.compute_r(A)
        r_2 = self.compute_r(B)
        r_2_reduced = self.compute_r(B_reduced)
        r_u_full = self.compute_r(U)
        r_u_reduced = self.compute_r(U_reduced)


        self.z_1, self.c_1 = self.optimize_z_1_c_1(r_1, r_u_reduced, r_2_reduced)
        self.z_2, self.c_2 = self.optimize_z_2_c_2(r_2, r_u_full, r_1)

        return self.z_1, self.c_1, self.z_2, self.c_2, U

    def predict(self, X, z_1, c_1, z_2, c_2):
        """
        Make predictions using the fitted UQTSVM model.
        X: Test data (features).
        """
        y_hat = np.ones(X.shape[0])
        for i in range(X.shape[0]):
            r_i = self.compute_r(X[i,:].reshape(1,-1))
            H_i = self.compute_H(X[i,:])

            numerator_1 = np.abs(r_i @ z_1 + c_1)
            denominator_1 = np.linalg.norm(H_i @ z_1 + c_1) ** 2
            numerator_2 = np.abs(r_i @ z_2 + c_2)
            denominator_2 = np.linalg.norm(H_i @ z_2 + c_2) ** 2

            expression_1 = numerator_1 / denominator_1
            expression_2 = numerator_2 / denominator_2

            if expression_1 >= expression_2:
                y_hat[i] = -1

        return y_hat

    def score(self, X, y):
        """
        Calculate the accuracy of the U-QSVM model.

        X: Test data (features).
        y: True labels for the test data.
        """
        #predictions = self.predict(X, z_1, c_1, z_2, c_2)
        predictions = self.predict(X, self.z_1, self.c_1, self.z_2, self.c_2)

        return accuracy_score(y, predictions)


    def compute_H(self, x):
      n = x.shape[0]
      s_x = self.compute_s_i(x)
      r_x = np.concatenate([s_x, x])
      X_x = np.kron(np.eye(n), x.reshape(1, -1))
      D_n = self.get_duplication_matrix(n)
      M_x = X_x @ D_n
      H_x = np.hstack([M_x, np.eye(n)])
      return H_x


    def optimize_z_1_c_1(self, r_1, r_u, r_2):

        d = r_1.shape[1]
        z_1 = cp.Variable((d, 1))
        c_1 = cp.Variable()

        # Square loss for regression points
        square_loss_1 = cp.sum_squares(r_1 @ z_1 + c_1)

        # Slack variables for errors in class 2 and Universum
        slack_class_2 = cp.Variable((r_2.shape[0], 1), nonneg=True)
        slack_universum = cp.Variable((r_u.shape[0], 1), nonneg=True)

        # slack_class_2 = cp.Variable(r_2.shape[0], nonneg=True)
        # slack_universum = cp.Variable(r_u.shape[0], nonneg=True)

        # Constraints
        constraints = []

        # Ensure all points in class 2 satisfy the margin or account for errors
        constraints.append(r_2 @ z_1 + c_1 <= -1 + slack_class_2)

        # Ensure all Universum points satisfy the margin with epsilon tolerance
        constraints.append(r_u @ z_1 + c_1 >= -1 + self.epsilon - slack_universum)

        # Objective: Minimize square loss + hinge penalties for class 2 and Universum
        objective = cp.Minimize(
            square_loss_1
            + C_1 * cp.sum(slack_class_2)
            + C_u * cp.sum(slack_universum)
        )

        # Problem definition
        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.SCS, verbose=False)  # Use any preferred solver

        return z_1.value, c_1.value


    def optimize_z_2_c_2(self, r_2, r_u, r_1):

        d = r_2.shape[1]
        z_2 = cp.Variable((d, 1))
        c_2 = cp.Variable()

        # Square loss for class 1 points
        square_loss_1 = cp.sum_squares(r_2 @ z_2 + c_2)

        # Slack variables for errors in class 1 and Universum
        slack_class_1 = cp.Variable((r_1.shape[0], 1), nonneg=True)
        slack_universum = cp.Variable((r_u.shape[0], 1), nonneg=True)

        # slack_class_1 = cp.Variable(r_1.shape[0], nonneg=True)
        # slack_universum = cp.Variable(r_u.shape[0], nonneg=True)

        # Constraints
        constraints = []

        # Ensure all points in class 1 satisfy the margin or account for errors
        constraints.append(r_1 @ z_2 + c_2 >= 1 - slack_class_1)

        # Ensure all Universum points satisfy the margin with epsilon tolerance
        constraints.append(r_u @ z_2 + c_2 >= 1 - self.epsilon - slack_universum)

        # Objective: Minimize square loss + hinge penalties for class 1 and Universum
        objective = cp.Minimize(
            square_loss_1
            + C_2 * cp.sum(slack_class_1)
            + C_u * cp.sum(slack_universum)
        )

        # Problem definition
        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.SCS, verbose=False)  # Use any preferred solver

        return z_2.value, c_2.value

    def compute_r(self, X):
        """Helper to compute the r matrix for predictions."""
        m, n = X.shape
        r = np.zeros((m, self.d))
        for i in range(m):
            x_i = X[i, :]
            s_i = self.compute_s_i(x_i)
            r[i] = np.concatenate([s_i, x_i])
        return r

    def compute_s_i(self, x_i):
        """Helper to compute s_i (half-vectorization) from x_i."""
        x_i_x_i_T = np.outer(x_i, x_i)
        return 0.5 * self.hvec(x_i_x_i_T)

    def hvec(self, A):
        """Half-vectorization of a symmetric matrix A."""
        indices = np.triu_indices(A.shape[0])
        return A[indices]

    def get_duplication_matrix(self, n):
        """Duplication matrix for vectorizing a symmetric matrix."""
        i_indices = np.arange(n * n)
        j_indices = np.zeros((n, n), dtype=int)

        z = 0
        for x in range(n):
            for y in range(x, n):
                j_indices[x, y] = j_indices[y, x] = z
                z += 1

        j_indices_flat = j_indices.flatten()
        data = np.ones(n * n, dtype=int)
        return coo_matrix((data, (i_indices, j_indices_flat)), shape=(n * n, (n * (n + 1)) // 2))

    def _calculate_d(self, n):
        """Helper to calculate the dimensionality once."""
        return n + n * (n + 1) // 2


In [67]:
C_1 = 1
C_2 = 1
C_u = 0.5
epsilon = 0.1
n_universum = 1289

uqtsvm = UQTSVM(C_1 = C_1, C_2 = C_2, C_u = C_u, epsilon = epsilon)
z_1, c_1, z_2, c_2, U = uqtsvm.fit(X_train_reduced, y_train, n_universum=1289)

In [68]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the test data
y_pred = uqtsvm.predict(X_train_reduced, z_1, c_1, z_2, c_2)

# Calculate the accuracy
accuracy = accuracy_score(y_train, y_pred)

# Output the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

# Display the classification report
print("Classification Report:")
print(classification_report(y_train, y_pred))

Accuracy: 93.58%
Classification Report:
              precision    recall  f1-score   support

          -1       0.96      0.97      0.96      1033
           1       0.79      0.73      0.76       167

    accuracy                           0.94      1200
   macro avg       0.87      0.85      0.86      1200
weighted avg       0.93      0.94      0.93      1200

